# Instalar zigbee2mqtt

Con [zigbee2mqtt](https://www.zigbee2mqtt.io/) podemos tener una red zigbee y usarla en home assistant mediante el protocolo mqtt

## Controlador o dongle

En primer lugar necesitamos tener un controlador que se conectará a la raspberry, en mi caso tengo el `SONOFF Zigbee 3.0 USB Dongle`

## Instalación de zigbee2mqtt con docker

En la página de [instalación con docker](https://www.zigbee2mqtt.io/guide/installation/02_docker.html) están las instrucciones

### Actualización de paquetes para raspberry

Si vamos a instalarlo en una raspberry primero hay que instalar unos paquetes

``` bash
sudo apt-key adv --keyserver hkps://keyserver.ubuntu.com:443 --recv-keys 04EE7237B7D453EC 648ACFD622F3D138
echo "deb http://httpredir.debian.org/debian buster-backports main contrib non-free" | sudo tee -a "/etc/apt/sources.list.d/debian-backports.list"
sudo apt update
sudo apt install libseccomp2 -t buster-backports
```

### Carpetas para el contenedor

Ahora nos creamos las carpetas necesarias para el contenedor

``` bash
sudo mkdir /docker/zigbee2mqtt
sudo mkdir /docker/zigbee2mqtt/data
```

### Encontrar la ruta del controlador

Para encontrar la ruta del controlador ejecutar

``` bash
ls -l /dev/serial/by-path/
```

En mi caso aparece `lrwxrwxrwx 1 root root 13 Aug 23 05:46 platform-fd500000.pcie-pci-0000:01:00.0-usb-0:1.1:1.0-port0 -> ../../ttyUSB0`, por lo que sé que su path es `/dev/ttyUSB0`

### Crear archivo de configuración

Crear el archivo de configuración

``` bash
sudo touch /docker/zigbee2mqtt/data/configuration.yaml
```

Lo abrimos con nano

``` bash

```

Y escribimos lo siguiente

``` bash
homeassistant: true

permit_join: true

mqtt:
  server: mqtt://mqtt:1883
  base_topic: zigbee2mqtt
  # Optional: MQTT server authentication user (default: nothing)
  # user: <USER>
  # Optional: MQTT server authentication password (default: nothing)
  # password: <PASSWORD>

serial:
  port: /dev/ttyACM0

frontend: true

advanced:
  homeassistant_legacy_entity_attributes: false
  legacy_api: false
  legacy_availability_payload: false
  network_key: GENERATE

device_options:
  legacy: false
```

### docker compose

Ir a portainer y en el stack de domotica añadir

``` bash
  zigbee2mqtt:
    container_name: zigbee2mqtt
    image: koenkk/zigbee2mqtt
    restart: unless-stopped
    volumes:
      - /docker/zigbee2mqtt/data:/app/data
      - /run/udev:/run/udev:ro
    ports:
      - 8080:8080
    environment:
      - TZ=Europe/Madrid
    devices:
      - /dev/ttyUSB0:/dev/ttyACM0
```

Y darle a deploy

Una vez levantado el contenedor podemos ir a `http://<IP>:8080/#/` para ver la red zigbee